We want to do a preliminary analysis of the grievances and disciplinary data to see if there is a smoke signal in the discipline data. 

We'll use the fuzzywuzzy string matching functions to match similar names. 

In [1]:
import os
import json
import pandas as pd
import altair as alt
from fuzzywuzzy import process

cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data')
source_dir = os.path.join(data_dir,'source')
manual_dir = os.path.join(data_dir, 'manual')
processed_dir = os.path.join(data_dir, 'processed')

# the discipline data that is one-row-per-incident
incident_xlsx = os.path.join(source_dir, '2010-2020.xlsx')
incident_df = pd.read_excel(
    incident_xlsx,
    parse_dates = ['FINAL DISP DATE']
)

# grievance data has been manually cleaned 
grievance_csv = os.path.join(manual_dir, 'officer_grievances_cleaned.csv')
grievance_df = pd.read_csv(
    grievance_csv,
    parse_dates = ['date']
)



In [233]:
grievance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   important_grievance  90 non-null     int64         
 1   control_number       89 non-null     object        
 2   date                 88 non-null     datetime64[ns]
 3   last_name            90 non-null     object        
 4   first_name           89 non-null     object        
 5   gender               82 non-null     object        
 6   race                 82 non-null     object        
 7   grievance            90 non-null     object        
 8   status               76 non-null     object        
 9   second_stage         40 non-null     object        
 10  notes                11 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 7.9+ KB


In [2]:
incident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12896 entries, 0 to 12895
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CONTROL #            12895 non-null  object        
 1   FINAL DISP DATE      12896 non-null  datetime64[ns]
 2   FINAL DISPOSITION    12896 non-null  object        
 3   FINAL # DAYS         2669 non-null   float64       
 4   EMPLOYEE LAST NAME   12895 non-null  object        
 5   EMPLOYEE FIRST NAME  12854 non-null  object        
 6   ALLEGATION           12888 non-null  object        
 7   COMP SEX             9360 non-null   object        
 8   COMP RACE            9184 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 906.9+ KB


We need the full names to match

In [3]:
grievance_df['full_name'] = grievance_df['first_name'] + ' ' + grievance_df['last_name']

incident_df['full_name'] = incident_df['EMPLOYEE FIRST NAME'] + ' ' + incident_df['EMPLOYEE LAST NAME']

In [5]:
incident_df['full_name'].value_counts()

RONALD BRIGHT        44
UNKNOWN UNKNOWN      41
WILLIAM WHITWORTH    40
BRANDON EVANS        39
WILLIE REAVES        36
                     ..
ROBERT DEBERRY        1
MARVIN CHAVOUS        1
DAVID IMHOF           1
TIFFANY HANSEN        1
BRANDON MASON         1
Name: full_name, Length: 2508, dtype: int64

In [236]:
# There is a null full name in the grievance DF that causes a lot of consternation. We drop it because we can't use it. 

grievance_df = grievance_df.dropna(subset=['full_name']).copy()

In [237]:
def extract_best_match(name, list_incident_names = (incident_df.full_name.unique())):
    results = process.extract(
        name,
        list_incident_names,
        limit=10
    )
    
    best_result_score = results[0][1] # [0][1] gets the score of the best match for the name
    
    if best_result_score == 100: # indicates perfect match
        best_result_name = results[0][0]
        
        return best_result_name
    else:
        print(name)
        print(results)
        print('enter the index of the best match, n if no match:')
        best_result_index = input()
        
        if best_result_index == 'n':
            return "no discipline"
        else:
            best_result_name = results[int(best_result_index)][0]
            return best_result_name
    

In [238]:
# dropna because it causes errors in the fuzzywuzzy functions 
grievance_name_list = (grievance_df.full_name.dropna().unique())

# define the name of the json we'll save the matching-dictionary to. That way we'll only need to do it once
all_name_matches_json = os.path.join(processed_dir, 'all_name_matches.json')

# if the file doesn't exist, do the work to create it
if not os.path.isfile(all_name_matches_json):
    all_name_matches = {}

    for name in grievance_name_list:
        all_name_matches[name] = extract_best_match(name)
    
    with open(all_name_matches_json, 'w') as f:
        json.dump(all_name_matches, f)

# if the file does exist, load it as a dict
if os.path.isfile(all_name_matches_json):
    with open(all_name_matches_json, 'r') as f:
        all_name_matches = json.load(f)
    



In [239]:
grievance_df['name_match'] = grievance_df.apply(
    lambda x: all_name_matches[x['full_name']],
    axis=1
)

In [240]:
###

There are 80 officers on the grievance list. 20 of those officers (25%) have never faced any discipline. 

In [241]:
len(grievance_name_list)

80

In [242]:
grievance_df[['full_name', 'name_match']].drop_duplicates().name_match.value_counts()

no discipline      20
KEVIN COOLEY        1
NEIL COOK           1
LYNSIE SMITH        1
CYRETHA FRAZIER     1
                   ..
ROBERT HOWSE        1
JUSTIN COKER        1
DAVID STANLEY       1
DEVON POWELL        1
EDWARD HOLLIDAY     1
Name: name_match, Length: 61, dtype: int64





A more accurate measure would be only looking at the grievances that we consider important. 


35 officers filed grievances that we think may be relevant. 9 of those 35 (25.7%) never faced any discipline. 

In [243]:
important  = grievance_df[grievance_df.important_grievance==1].copy()

In [244]:
len(important.full_name.unique())

35

In [245]:
important[['full_name', 'name_match']].drop_duplicates().name_match.value_counts()

no discipline       9
DAVID STANLEY       1
SHARISSA PEREZ      1
CHRISTINE DAVIS     1
MONICA BLAKE        1
GBEWA MUSTAPHA      1
ROBERT HOWSE        1
ANITA LOCKRIDGE     1
TAYLOR HURLEY       1
VICTORIA LONG       1
KATHY LEWIS         1
ANGELA HERRERA      1
JULIE RIGGS         1
DEVON POWELL        1
MADISON MEISS       1
BRITTANY MCELWEE    1
CHRISTINE OLSON     1
CYRETHA FRAZIER     1
DHANA JONES         1
Valerie Wilks       1
CITLALY GOMEZ       1
ANTIGONA GASHI      1
PHILLIP SMITH       1
ANTHONY WILFERT     1
BARRETT TEAGUE      1
LYNSIE SMITH        1
LAURA WALL          1
Name: name_match, dtype: int64

In [246]:

# calculates the closest date to the incident date 
# operates on a per-row-basis
def calculate_closest_date(df):
    grievance_name = df.name_match
    grievance_date = df.date
    
    
    if grievance_name == 'no discipline':
        return pd.NaT
    else: 
        # get a list of all the discipline dates using the name match from extract_best_match
        incident_dates = incident_df[
            incident_df.full_name == grievance_name
        ]['FINAL DISP DATE'].to_list()
        
        # map a timedelta lambda over the list of discipline dates
        # the lamda takes the date of the discipline and subtracts the date of the grievance. 
        # That gives us the number of days from the grievance to the discipline 
        list_timedelta = list(
            map(
                lambda x: x - grievance_date,
                incident_dates
            )
        )
        
        # inline if-statement to get only the positive days (grievance happened before discipline)
        positive_list_timedelta = [i for i in list_timedelta if i.days >= 0]
        
        # no positive dates: no discipline after grievance, only before 
        if len(positive_list_timedelta) == 0:
            return pd.NaT
        
        # take the smallest number of days 
        else:
            return min(positive_list_timedelta)
    

In [253]:
important

,important_grievance,control_number,date,last_name,first_name,gender,race,grievance,status,second_stage,notes,full_name,name_match,time_to_discipline
0,1,11-01,2011-11-17,Long,Vickie,F,H,Denial of IOD Claim,Stage 1 - Conducting investigation,letter 12/13/11 - She was out on leave - IOD ...,NaN,Vickie Long,VICTORIA LONG,393 days
1,1,11-02,2011-11-18,Peterson,Michelle,NaN,NaN,Unprofessional conduct - Supervisor,Stage 1 -2 issues substantiated - 1 can not be...,letter 1/17/12,Resolved - 1/18/12,Michelle Peterson,no discipline,NaT
2,1,01-12,2012-01-18,Peterson,Michelle,NaN,NaN,Retaliation - unprofessional conduct,Investigating - Have an extenstion 1/18/12,Resolved 3-5-12,NaN,Michelle Peterson,no discipline,NaT
3,1,05-01,2012-05-21,Wilfert,Anthony,M,W,Discrimination/harassment,Claim cannot be substantiated - Stage 1 given ...,NaN,NaN,Anthony Wilfert,ANTHONY WILFERT,50 days
4,1,08-01,2012-08-15,Jordan,Jacqueline,F,B,Discrimination,"Turned over to OPA on Monday, August 20, 2012",NaN,NaN,Jacqueline Jordan,no discipline,NaT
5,1,08-02,2012-08-24,Lewis,Kathy,F,B,AWOL Status,Stage 1 - Letter - Resolution possible team tr...,NaN,NaN,Kathy Lewis,KATHY LEWIS,18 days
6,1,11-01,2012-11-07,Olson,Christine,F,W,Employee Relations,Stage 1 Letter - Not substantiated - 11-14-12,Did not appeal by 11-27-12- Case Closed,NaN,Christine Olson,CHRISTINE OLSON,496 days
7,1,1201,2016-12-16,Howse,Robert,M,B,unfair discipline,letter sent 1/13/17,NaN,NaN,Robert Howse,ROBERT HOWSE,108 days
8,1,201701-01,2017-01-03,James,Jasmine,F,B,harassment and workplace bullying,not sustained/ letter sent 3-22-17,NaN,NaN,Jasmine James,no discipline,NaT
9,1,201702-01,2017-02-21,Smith,Philip R.,M,W,Harassment,not sustained/ letter sent 5/8/17,NaN,NaN,Philip R. Smith,PHILLIP SMITH,NaT


In [248]:
important['time_to_discipline'] = important.apply(calculate_closest_date, axis=1)

In [255]:
len(important)

41

17 of the important grievances either had no discipline filed against the officer at all or no discipline post-grievance. 

24 of the important grievances _did_ face discipline after the fact, some as soon as 14 days 

In [249]:
important.time_to_discipline.value_counts(dropna=False)

NaT         17
88 days      2
78 days      2
140 days     1
18 days      1
14 days      1
478 days     1
266 days     1
205 days     1
496 days     1
253 days     1
153 days     1
442 days     1
25 days      1
96 days      1
108 days     1
294 days     1
26 days      1
306 days     1
393 days     1
19 days      1
50 days      1
93 days      1
Name: time_to_discipline, dtype: int64

In [250]:
important.time_to_discipline.value_counts().sum()

24

In [251]:
important.groupby('gender').time_to_discipline.agg(lambda x: x.mean())

gender
F   209 days 00:00:00
M    74 days 04:00:00
Name: time_to_discipline, dtype: timedelta64[ns]

In [252]:
important.groupby('race').time_to_discipline.agg(lambda x: x.mean())

race
B   139 days 10:54:32.727272728
H             264 days 06:00:00
W             179 days 13:20:00
Name: time_to_discipline, dtype: timedelta64[ns]

In [254]:
important.groupby(['gender','race']).time_to_discipline.agg(lambda x: x.mean())

gender  race
F       B      155 days 06:00:00
        H      264 days 06:00:00
        W      243 days 20:00:00
M       B       97 days 08:00:00
        W       51 days 00:00:00
Name: time_to_discipline, dtype: timedelta64[ns]